In [1]:
import json
import requests

In [2]:
search_template = """
{
  "query":{
    "function_score":{
      "functions":[
        {
          "exp":{
            "issued.date":{
              "offset":"30d",
              "scale":"1800d"
            }
          }
        }
      ],
      "score_mode":"sum",
      "query":{
        "bool":{
          "must":[
            {
              "match_all": {}
            }
          ],
          "filter":[
            {
              "bool":{
                "should":[
                  {
                    "term":{
                      "objectProvides":"FAQ"
                    }
                  }
                ],
                "minimum_should_match":1
              }
            },
            {
              "constant_score":{
                "filter":{
                  "bool":{
                    "should":[
                      {
                        "bool":{
                          "must_not":{
                            "exists":{
                              "field":"expires"
                            }
                          }
                        }
                      },
                      {
                        "range":{
                          "expires":{
                            "gte":"2022-03-16T15:59:50Z"
                          }
                        }
                      }
                    ]
                  }
                }
              }
            },
            {
              "bool":{
                "should":[
                  {
                    "range":{
                      "readingTime":{
                        
                      }
                    }
                  }
                ],
                "minimum_should_match":1
              }
            },
            {
              "bool":{
                "should":[
                  {
                    "range":{
                      "issued.date":{
                        
                      }
                    }
                  }
                ],
                "minimum_should_match":1
              }
            },
            {
              "bool":{
                "should":[
                  {
                    "term":{
                      "language":"en"
                    }
                  }
                ],
                "minimum_should_match":1
              }
            },
            {
              "term":{
                "hasWorkflowState":"published"
              }
            },
            {
              "constant_score":{
                "filter":{
                  "range":{
                    "issued.date":{
                      "lte":"2022-03-16T15:59:52Z"
                    }
                  }
                }
              }
            }
          ]
        }
      }
    }
  },
  "size":10000,
  "from":0,
  "track_total_hits":true,
  "explain":false
}
"""

In [3]:
import re

patterns = [
    re.compile("Do something for our planet, print this page only if needed. Even a small action can make an enormous difference when millions of people do it!"),
    re.compile("HelpCenter FAQ"),
    re.compile("PDF generated.*", flags=re.MULTILINE),
    re.compile("Filed under:.*", flags=re.MULTILINE),
    re.compile("The EEA is an agency of the European Union.*", re.MULTILINE)
]

def clean(obj):
    text = obj['fulltext']

    for pat in patterns:
        text = pat.sub("", text)        

    text = re.sub("Engineered by:.*(\n.*)*", "", text, flags=re.MULTILINE)
    #print('---[text]---')
    #print(text)
    #print('---[fulltext]---')
    #print(obj['fulltext'])
    obj['fulltext'] = text
    
    return obj

In [4]:
host = "http://10.120.10.131:57664/data_searchui/_search"

resp = requests.post(host, json=json.loads(search_template))
hits = resp.json()['hits']['hits']
hits = [clean(h['_source']) for h in hits]
questions = [{"question": h["title"], "answer": h["fulltext"], "id": h["id"]} for h in hits]

fname = f"data/faqs.json"

with open(fname, 'w') as f:
    json.dump(questions, f)
    
print(f"Generated {len(questions)} pairs")

Generated 59 pairs
